# IMPORTS

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.cluster import KMeans



## LEITURA DOS DADOS

In [16]:
df = pd.read_csv('../../RawData/train_radiomics_occipital_CONTROL.csv') 
x_train = df.drop(columns=['Transition'])

# Tratamento

### Single Value Colums 

In [ ]:
df = df.loc[:, df.nunique() > 1]

print(df.shape)
#  Analise das colunas que tem menos de 50 valores unicos 
n = df.nunique()

#for col, e in n.items():
#    if e < 50:  
#        print(f"Coluna: {col}, Valores Unicos : {e}")

(305, 2026)
diagnostics_Image-original_Mean        305
diagnostics_Image-original_Maximum      92
diagnostics_Mask-original_VoxelNum     300
diagnostics_Mask-original_VolumeNum      4
original_shape_Elongation              305
                                      ... 
lbp-3D-k_ngtdm_Contrast                305
lbp-3D-k_ngtdm_Strength                305
Sex                                      2
Age                                      4
Transition                               5
Length: 2026, dtype: int64


### AGE BINING  ( secalhar isto era no 2 )

In [18]:

age_bins = [0, 65, 75, 85, 100]
# BINS_SIZER = ['<65', '65-74', '75-84', '85+']
age_labels = [60, 70, 80, 90] # VALOR MEDIO DO BIN 
df['Age'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels).astype(int)

#print(df['Age'])

### Colunas Categoricas 

In [19]:
# Analisar a contagem de valores únicos para cada coluna categórica

# Identificar as colunas categóricas
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

#for col in categorical_columns:
#    print(f"\nColuna: {col}")

#colunas_catagoricas_a_remover = ['ID', 'Image', 'Mask', 'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash'] 

colunas_catagoricas_a_remover = ['ID', 'Image', 'Mask', 'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'] 

# ** Bounding Box
#
# ** as colunas do 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'
# ** Deveriam ser retiradas, mas o bounding box pode ser importante para a zona de maior ativação do Alzimeir 
# **  ja a de centro de maxima devem ser muito correlacionados, por isso devem ser retirados mais para a frente 
# *TODO acabei por retirar para correr melhor os modelos, mas analisar se se deve retirar ou nao 

df.drop(columns=colunas_catagoricas_a_remover,axis= 1 , inplace= True)

print(df.shape)


(305, 2026)


### Label Encoding

In [20]:
label_mapping = {
    'CN-CN': 0,
    'AD-AD': 1,
    'MCI-AD': 2,
    'MCI-MCI': 3,
    'CN-MCI' : 4
}

df['Transition'] = df['Transition'].map(label_mapping)
print(df.shape)
df.to_csv('1_Occipital_pre_processamento', index=False) 

(305, 2026)


# Models

In [21]:

#experiment = setup(df, target='Transition')

In [22]:
#best = compare_models()

Com o tratamento atual, o pycaret diz que os melhores modelos são :
- Random Forest Classifier
- Extra Trees Classifier
- Gradient Boosting Classifier 
- K Neighbors Classifier
- Extreme Gradient Boosting 


### Aplicação dos modelos (default) ao dataset tratado

In [23]:


X = df.drop('Transition', axis=1)
y = df['Transition']


# Dividir o conjunto de dados em treino e teste com estratificação
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=34, stratify=y)


# Inicializar dicionário para armazenar as pontuações de cross-validation do modelo
model_cross_score = {}


### Funcao Matrix de Confusao 

In [24]:
def plot_confusion_matrix_with_labels(confusion_matrix, label_mapping):
    # Criar um mapeamento inverso
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}
    
    # Aplicar o mapeamento inverso na matriz de confusão
    cm_with_labels = np.zeros_like(confusion_matrix, dtype=object)
    for i in range(confusion_matrix.shape[0]):
        for j in range(confusion_matrix.shape[1]):
            cm_with_labels[i, j] = f"{reverse_label_mapping[i]} (Pred: {reverse_label_mapping[j]})"

    # Criar um DataFrame para facilitar a visualização
    df_cm = pd.DataFrame(confusion_matrix, index=reverse_label_mapping.values(), columns=reverse_label_mapping.values())
    
    # Plotar a matriz de confusão
    plt.figure(figsize=(10, 7))
    sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix with Labels')
    plt.show()

#### Random Forest Classifier

In [25]:

rf_model = RandomForestClassifier(n_estimators=100, max_features='sqrt', random_state=34)

rf_score = cross_val_score(rf_model, X_train, y_train, cv=10,scoring='f1_macro')
print(rf_score)

model_cross_score['random_forest'] = rf_score





/home/cid34senhas/miniconda3/envs/DAA123/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


[0.20714286 0.31859649 0.33333333 0.23939394 0.17619048 0.33182957
 0.24606061 0.34398778 0.21590909 0.39027778]


#### Extra Trees Classifier

In [26]:
et_model = ExtraTreesClassifier(criterion='gini', max_depth=20, random_state=34)

et_score = cross_val_score(et_model, X_train, y_train, cv=10,scoring='f1_macro')

print(et_score)

model_cross_score['Extra Trees'] = et_score

/home/cid34senhas/miniconda3/envs/DAA123/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


[0.14607843 0.26754386 0.26147186 0.24646465 0.19428571 0.2248366
 0.14285714 0.18       0.24358974 0.4       ]


#### Gradient Boosting Classifier 

In [27]:
gb_model = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, random_state=34)

score_cross = cross_val_score(gb_model,X_train,y_train,cv=10,scoring='f1_macro')

print(score_cross)

model_cross_score['Gradient_boosting'] = score_cross



/home/cid34senhas/miniconda3/envs/DAA123/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(


KeyboardInterrupt: 

#### K Neighbors Classifier

In [ ]:
kn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto')
k_night_croos_score = cross_val_score(kn_model, X_train, y_train, cv=10,scoring='f1_macro')

print("Acuracy k night", k_night_croos_score)


model_cross_score['k_night'] = k_night_croos_score




#### Extreme Gradient Boosting

In [ ]:
xgb_model = XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.05,colsample_bytree=0.4, subsample=0.8, random_state=34)


score_cross = cross_val_score(xgb_model,X_train,y_train,cv=10,scoring='f1_macro')

print(score_cross)

model_cross_score['XGBoost'] = score_cross



SVC

In [ ]:


svc_moldel = SVC(kernel='rbf', random_state=34)


score_cross = cross_val_score(svc_moldel,X_train,y_train,cv=10,scoring='f1_macro')

print(score_cross)

model_cross_score['SVC'] = score_cross



In [ ]:
def print_model_scores(model_scores):
    for model, scores in model_scores.items():
        max_score = np.max(scores)
        min_score = np.min(scores)
        diff = max_score - min_score
        avg_score = np.mean(scores)
        
        print(f"{model} (Max: {max_score:.4f}, Min: {min_score:.4f}, Diff: {diff:.4f}):")
        print(f"  Scores: {', '.join([f'{score:.4f}' for score in scores])}")
        print(f"  Average: {avg_score:.4f}")
        print('-' * 50)

# Chamar a função para imprimir o dicionário formatado
print_model_scores(model_cross_score)